In [ ]:
import numpy as np
import h5py
import cleo_tools as cleo
import time 

import matplotlib.pylab as plt
%matplotlib notebook

import seaborn as sn

In [ ]:
infile = open('small_CLEO_test_file.dat')
#infile = open('data31_100k_LARGE.dat')
collisions = cleo.get_collisions(infile)

In [ ]:
lp = []
lk = []
lm = []
le = []
lph = []

pis = []
kas = []
mus = []
els = []
phs = []

for collision in collisions:
    
    pions,kaons,muons,electrons,photons = collision
    npi = len(pions)
    nka = len(kaons)
    nmu = len(muons)
    nel = len(electrons)
    nph = len(photons)
    
    lp.append(npi)
    lk.append(nka)
    lm.append(nmu)
    le.append(nel)
    lph.append(nph)
    
    for pion in pions:
        pis.append(pion)
    
    for kaon in kaons:
        kas.append(kaon)
        
    for muon in muons:
        mus.append(muon)
    
    for electron in electrons:
        els.append(electron)
    
    for photon in photons:
        phs.append(photon)

        
entries = ["pions","kaons","muons","electrons","photons"]

In [ ]:
alg = 'lzf'

hf = h5py.File('self_defined_cleo.hdf5','w')
#hf.close()
#'''
hf.create_dataset('entries',data=entries)

hf.create_dataset('npions',data=lp,compression="gzip", compression_opts=9)
hf.create_dataset('nkaons',data=lk,compression="gzip", compression_opts=9)
hf.create_dataset('nmuons',data=lm,compression="gzip", compression_opts=9)
hf.create_dataset('nelectrons',data=le,compression="gzip", compression_opts=9)
hf.create_dataset('nphotons',data=lph,compression="gzip", compression_opts=9)

g1 = hf.create_group('particles')

g1.create_dataset('pions',data=pis,compression="gzip", compression_opts=9)
g1.create_dataset('kaons',data=kas,compression="gzip", compression_opts=9)
g1.create_dataset('muons',data=mus,compression="gzip", compression_opts=9)
g1.create_dataset('electrons',data=els,compression="gzip", compression_opts=9)
g1.create_dataset('photons',data=phs,compression="gzip", compression_opts=9)

hf.close()
#'''

In [ ]:
#hf.close()

In [ ]:
#hf = h5py.File('self_defined_cleo.hdf5','r') 

In [ ]:
#entries = hf.get("entries")

In [ ]:
#entries.value[0]

In [ ]:
def get_entries(filename,entry_names):
    
    hf = h5py.File(filename,'r') 
    
    entries = {}
    
    for name in entry_names:
        num_name = "n%s" % (name)
        entries[num_name] = hf.get(num_name).value
        particle_name = "particles/%s" % (name)
        entries[name] = hf.get(particle_name).value
        
    hf.close()
        
    return entries

In [ ]:
data = get_entries('self_defined_cleo.hdf5',entries)

In [ ]:
e = data["pions"].transpose()[0]

In [ ]:
plt.figure()
plt.hist(e,bins=100,range=(0,1));

In [ ]:
def close_all_hdf_files():
    import gc
    for obj in gc.get_objects():   # Browse through ALL objects
        if isinstance(obj, h5py.File):   # Just HDF5 files
            try:
                obj.close()
            except:
                pass # Was already closed

In [ ]:
close_all_hdf_files()

In [ ]:
pis_trans = np.array(pis).transpose()
e = pis_trans[0]
px = pis_trans[1]
py = pis_trans[2]
pz = pis_trans[3]
q = pis_trans[4]

In [ ]:
# More complicated version
#h5py.File('self_defined_cleo_MORE_COMPLICATED.hdf5','r').close()
#hf.close()
with h5py.File('self_defined_cleo_MORE_COMPLICATED.hdf5','w',) as hf:
    #hf.close()
    #'''
    hf.create_dataset('entries',data=entries)

    groupnames = ["pions","kaons","muons","electrons","photons"]
    groupvars = [pis,kas,mus,els,phs]
    numbers = [lp,lk,lm,le,lph]
    fields = ["e","px","py","pz","q"]

    for groupname,groupvar,number in zip(groupnames,groupvars,numbers):

        g = hf.create_group(groupname)

        trans = np.array(groupvar).transpose()
        e = trans[0]
        px = trans[1]
        py = trans[2]
        pz = trans[3]
        
        if groupname is not "photons":
            q = trans[4]

        g.create_dataset("e",data=e,compression="gzip", compression_opts=9)
        g.create_dataset("px",data=px,compression="gzip", compression_opts=9)
        g.create_dataset("py",data=py,compression="gzip", compression_opts=9)
        g.create_dataset("pz",data=pz,compression="gzip", compression_opts=9)
        
        if groupname is not "photons":
            g.create_dataset("q",data=q,compression="gzip", compression_opts=9)

        g.create_dataset("n",data=number,compression="gzip", compression_opts=9)

        del trans

    hf.close()

In [ ]:
#close_all_hdf_files()

In [ ]:
#len(pis_trans[0])

In [ ]:
with h5py.File('self_defined_cleo_MORE_COMPLICATED.hdf5','r') as hf:
    
    e = hf.get("pions/e").value


In [ ]:
print len(e)
plt.figure()
plt.hist(e,bins=100,range=(0,1));

In [ ]:
def get_entries_complicated(filename,entry_names):

    fields = ["n","e","px","py","pz","q"]

    entries = {}
    
    with h5py.File(filename,'r') as hf:
        for name in entry_names:

            sub_entry = {}

            for field in fields:
                entry_name = "%s/%s" % (name,field)
                #key_name = "%s.%s" % (name,field)
                #print entry_name
                sub_entry[field] = hf.get(entry_name).value
            entries[name] = sub_entry
        hf.close()
        
    return entries

In [ ]:
values = get_entries_complicated("self_defined_cleo_MORE_COMPLICATED.hdf5",["pions"])

In [ ]:
values.keys()

In [ ]:
e = values["pions"]["pz"]
plt.figure()
plt.hist(e,bins=100,range=(0,1));

In [ ]:
def get_event(values,nevent=0,return_dicts=False):
    
    keys = values.keys()
    
    ret = {}
    
    for key in keys:
        ret[key] = {}
        print key
        
        n = values[key]["n"]
        start = sum(n[0:nevent])
        end = start + n[nevent]
        print start,end

        sub_keys = values[key].keys()
        sub_keys.remove("n")
        
        if return_dicts:
            for sk in sub_keys:
                ret[key][sk] = values[key][sk][start:end]        
        else:
            vals = []
            for sk in sub_keys:
                vals.append(values[key][sk][start:end])
            vals = np.array(vals).transpose()
            ret[key] = vals
                            
    return ret

In [ ]:
x = get_event(values,11,return_dicts=True)

In [ ]:
x["pions"]

In [ ]:
x = get_event(values,11)

In [ ]:
x["pions"]

In [ ]:
for pion in x["pions"]:
    e,px,py,pz,q = pion
    print pion